<a href="https://colab.research.google.com/github/ZukowskaAnna/MEDICA/blob/main/SystemMedicaPlus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install Flask pyngrok



In [ ]:
import sqlite3
from flask import Flask, request, redirect, url_for, session, render_template_string # Użyjemy render_template_string dla prostoty w Colab
import os
from pyngrok import ngrok

# Konfiguracja Flask
app = Flask(__name__)
# Ważne dla bezpieczeństwa sesji! W prawdziwej aplikacji użyj czegoś bardziej skomplikowanego i trzymaj to w sekrecie.
app.secret_key = os.urandom(24)

# Nazwa pliku bazy danych
DATABASE = 'medica_plus.db'

# Funkcja do połączenia z bazą danych
def get_db():
    conn = sqlite3.connect(DATABASE)
    conn.row_factory = sqlite3.Row # Zwraca wiersze jako obiekty podobne do słowników
    return conn

# Funkcja do inicjalizacji bazy danych (tworzenie tabeli)
def init_db():
    conn = get_db()
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS patients (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            imie TEXT NOT NULL,
            nazwisko TEXT NOT NULL,
            pesel TEXT NOT NULL UNIQUE, -- PESEL jako unikalny identyfikator do logowania
            password TEXT NOT NULL -- WAŻNE: W prawdziwej aplikacji hasła MUSZĄ być hashowane!
            -- Można dodać więcej pól zgodnie z potrzebami (np. data urodzenia, adres, etc.)
        )
    ''')
    conn.commit()
    conn.close()
    print("Baza danych zainicjalizowana.")

# Inicjalizacja bazy danych przy starcie
init_db()

# Konfiguracja ngrok (dla udostępnienia aplikacji z Colab)
# Zarejestruj się na ngrok.com, aby uzyskać authtoken, jeśli chcesz mieć stabilniejsze tunele
# ngrok.set_auth_token("TWÓJ_NGROK_AUTHTOKEN") # Opcjonalnie

Baza danych zainicjalizowana.


In [ ]:
# Szablon HTML dla formularza rejestracji
REGISTER_FORM_HTML = """
<!DOCTYPE html>
<html>
<head>
    <title>Rejestracja Pacjenta - Medica+</title>
    <style>
        body { font-family: sans-serif; margin: 20px; }
        form { max-width: 400px; padding: 20px; border: 1px solid #ccc; border-radius: 5px; }
        label, input { display: block; margin-bottom: 10px; }
        input[type="text"], input[type="password"] { width: calc(100% - 22px); padding: 10px; }
        input[type="submit"] { padding: 10px 15px; background-color: #007bff; color: white; border: none; cursor: pointer; }
        .error { color: red; }
        a { display: block; margin-top: 15px; }
    </style>
</head>
<body>
    <h2>Rejestracja Nowego Pacjenta</h2>
    {% if error %}
        <p class="error">{{ error }}</p>
    {% endif %}
    <form method="post">
        <label for="imie">Imię:</label>
        <input type="text" id="imie" name="imie" required>

        <label for="nazwisko">Nazwisko:</label>
        <input type="text" id="nazwisko" name="nazwisko" required>

        <label for="pesel">PESEL:</label>
        <input type="text" id="pesel" name="pesel" required pattern="\d{11}" title="PESEL musi składać się z 11 cyfr">

        <label for="password">Hasło:</label>
        <input type="password" id="password" name="password" required>

        <input type="submit" value="Zarejestruj">
    </form>
    <a href="{{ url_for('login') }}">Masz już konto? Zaloguj się</a>
</body>
</html>
"""

@app.route('/register', methods=['GET', 'POST'])
def register():
    error = None
    if request.method == 'POST':
        imie = request.form['imie']
        nazwisko = request.form['nazwisko']
        pesel = request.form['pesel']
        password = request.form['password'] # PAMIĘTAJ: Hashuj hasła w realnej aplikacji!

        conn = get_db()
        cursor = conn.cursor()
        try:
            cursor.execute(
                "INSERT INTO patients (imie, nazwisko, pesel, password) VALUES (?, ?, ?, ?)",
                (imie, nazwisko, pesel, password)
            )
            conn.commit()
            conn.close()
            # Po pomyślnej rejestracji przekieruj na stronę logowania
            return redirect(url_for('login'))
        except sqlite3.IntegrityError:
            # Obsługa błędu, jeśli PESEL już istnieje
            error = "Pacjent o podanym numerze PESEL już istnieje."
            conn.close()
        except Exception as e:
            error = f"Wystąpił błąd: {e}"
            conn.rollback() # Wycofaj zmiany w razie innego błędu
            conn.close()

    # Dla metody GET lub jeśli wystąpił błąd POST, wyświetl formularz
    return render_template_string(REGISTER_FORM_HTML, error=error)

In [ ]:
# Szablon HTML dla formularza logowania
LOGIN_FORM_HTML = """
<!DOCTYPE html>
<html>
<head>
    <title>Logowanie - Medica+</title>
     <style>
        body { font-family: sans-serif; margin: 20px; }
        form { max-width: 400px; padding: 20px; border: 1px solid #ccc; border-radius: 5px; }
        label, input { display: block; margin-bottom: 10px; }
        input[type="text"], input[type="password"] { width: calc(100% - 22px); padding: 10px; }
        input[type="submit"] { padding: 10px 15px; background-color: #28a745; color: white; border: none; cursor: pointer; }
         .error { color: red; }
        a { display: block; margin-top: 15px; }
    </style>
</head>
<body>
    <h2>Logowanie Pacjenta</h2>
    {% if error %}
        <p class="error">{{ error }}</p>
    {% endif %}
    <form method="post">
        <label for="pesel">PESEL:</label>
        <input type="text" id="pesel" name="pesel" required>

        <label for="password">Hasło:</label>
        <input type="password" id="password" name="password" required>

        <input type="submit" value="Zaloguj">
    </form>
    <a href="{{ url_for('register') }}">Nie masz konta? Zarejestruj się</a>
</body>
</html>
"""

@app.route('/login', methods=['GET', 'POST'])
def login():
    error = None
    if request.method == 'POST':
        pesel = request.form['pesel']
        password = request.form['password']

        conn = get_db()
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM patients WHERE pesel = ?", (pesel,))
        patient = cursor.fetchone()
        conn.close()

        if patient:
            # Weryfikacja hasła (w realnej aplikacji porównaj hashe!)
            if patient['password'] == password:
                session['patient_id'] = patient['id'] # Zapisz ID pacjenta w sesji
                session['patient_pesel'] = patient['pesel'] # Można też zapisać PESEL lub inne dane
                return redirect(url_for('patient_card')) # Przekieruj do karty pacjenta
            else:
                error = "Nieprawidłowe hasło."
        else:
            error = "Nie znaleziono pacjenta o podanym numerze PESEL."

    # Dla GET lub błędu logowania, wyświetl formularz
    return render_template_string(LOGIN_FORM_HTML, error=error)

In [ ]:
# Szablon HTML dla Karty Pacjenta
PATIENT_CARD_HTML = """
<!DOCTYPE html>
<html>
<head>
    <title>Karta Pacjenta - Medica+</title>
    <style>
        body { font-family: sans-serif; margin: 20px; }
        .card { border: 1px solid #ccc; border-radius: 5px; padding: 20px; max-width: 500px; }
        .card h2 { margin-top: 0; }
        .card p { margin: 5px 0; }
        .card strong { display: inline-block; width: 100px; }
        a { display: inline-block; margin-top: 15px; padding: 8px 12px; background-color: #dc3545; color: white; text-decoration: none; border-radius: 3px;}
    </style>
</head>
<body>
    <div class="card">
        <h2>Karta Pacjenta</h2>
        {% if patient %}
            <p><strong>Imię:</strong> {{ patient['imie'] }}</p>
            <p><strong>Nazwisko:</strong> {{ patient['nazwisko'] }}</p>
            <p><strong>PESEL:</strong> {{ patient['pesel'] }}</p>
            <p><i>(Tu w przyszłości pojawi się historia wizyt, wyniki badań, itp.)</i></p>
        {% else %}
            <p>Nie można załadować danych pacjenta.</p>
        {% endif %}
        <a href="{{ url_for('logout') }}">Wyloguj</a>
    </div>

</body>
</html>
"""

@app.route('/patient_card')
def patient_card():
    # Sprawdź, czy pacjent jest zalogowany (czy jego ID jest w sesji)
    if 'patient_id' not in session:
        return redirect(url_for('login')) # Jeśli nie, przekieruj do logowania

    patient_id = session['patient_id']
    conn = get_db()
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM patients WHERE id = ?", (patient_id,))
    patient_data = cursor.fetchone()
    conn.close()

    if not patient_data:
         # Jeśli z jakiegoś powodu nie ma danych, mimo że ID jest w sesji
         session.pop('patient_id', None) # Wyczyść sesję
         return redirect(url_for('login'))

    return render_template_string(PATIENT_CARD_HTML, patient=patient_data)


@app.route('/logout')
def logout():
    # Usuń dane pacjenta z sesji
    session.pop('patient_id', None)
    session.pop('patient_pesel', None)
    return redirect(url_for('login')) # Przekieruj do strony logowania

In [ ]:
# Importuj ngrok, jeśli nie zostało to zrobione globalnie wcześniej w tej komórce
from pyngrok import ngrok
# Upewnij się, że zmienna 'app' (twoja aplikacja Flask) jest zdefiniowana w poprzednich komórkach

# === POCZĄTEK SEKCJI KONFIGURACJI NGROK ===

# Wklej tutaj swój authtoken skopiowany z panelu Ngrok: https://dashboard.ngrok.com/get-started/your-authtoken
# Zastąp "TWOJ_SKOPIOWANY_AUTHTOKEN" swoim rzeczywistym tokenem.
# Upewnij się, że token jest w cudzysłowach "".
AUTHTOKEN = "2vV3LkmdRsYIgHqIscMDgQtBWWU_5JztgMVb6RZ6fFpFKZj2U"

if not AUTHTOKEN:
  print("!!!!!!!!!!!!!!")
  print("BŁĄD: Nie wprowadzono authtoken Ngrok.")
  print("Przejdź na https://dashboard.ngrok.com/get-started/your-authtoken, skopiuj token i wklej go w kodzie poniżej.")
  print("!!!!!!!!!!!!!!")
  # Możesz tu rzucić wyjątek, żeby zatrzymać wykonanie, jeśli token nie jest ustawiony
  # raise ValueError("Ngrok Authtoken nie został ustawiony!")
else:
  ngrok.set_auth_token(AUTHTOKEN)

  # === KONIEC SEKCJI KONFIGURACJI NGROK ===


  # === SEKCJA URUCHAMIANIA APLIKACJI ===

  try:
    # Uruchomienie tunelu ngrok na porcie 5000 (domyślny port Flask)
    public_url = ngrok.connect(5000)

    print("====================================================================")
    print(f" * Aplikacja Medica+ jest dostępna publicznie pod adresem: {public_url}")
    print("====================================================================")
    print(f" * Link do rejestracji: {public_url}/register")
    print(f" * Link do logowania: {public_url}/login")
    print("--------------------------------------------------------------------")
    print("Serwer Flask startuje poniżej. Możesz zobaczyć logi zapytań.")
    print("(Aby zatrzymać serwer, musisz przerwać wykonywanie tej komórki w Colab)")
    print("--------------------------------------------------------------------")


    # Uruchomienie aplikacji Flask
    # Używamy use_reloader=False, ponieważ automatyczne przeładowywanie może sprawiać problemy w Colab/ngrok
    # Uruchomienie w __main__ jest technicznie niekonieczne w komórce Colab, ale to dobra praktyka
    if __name__ == '__main__':
        app.run(port=5000, use_reloader=False)

  except Exception as e:
    print("!!!!!!!!!!!!!!")
    print(f"Wystąpił błąd podczas uruchamiania ngrok lub Flask: {e}")
    print("Sprawdź, czy:")
    print("  1. Twój authtoken Ngrok jest poprawny i aktywny.")
    print("  2. Port 5000 nie jest już zajęty (choć w Colab to rzadkie).")
    print("  3. Aplikacja Flask ('app') została poprawnie zdefiniowana w poprzednich komórkach.")
    print("!!!!!!!!!!!!!!")

    # Opcjonalnie: Rozłącz istniejące tunele ngrok, jeśli wystąpił błąd po ich częściowym utworzeniu
    # for tunnel in ngrok.get_tunnels():
    #    ngrok.disconnect(tunnel.public_url)

 * Aplikacja Medica+ jest dostępna publicznie pod adresem: NgrokTunnel: "https://553b-35-189-171-218.ngrok-free.app" -> "http://localhost:5000"
 * Link do rejestracji: NgrokTunnel: "https://553b-35-189-171-218.ngrok-free.app" -> "http://localhost:5000"/register
 * Link do logowania: NgrokTunnel: "https://553b-35-189-171-218.ngrok-free.app" -> "http://localhost:5000"/login
--------------------------------------------------------------------
Serwer Flask startuje poniżej. Możesz zobaczyć logi zapytań.
(Aby zatrzymać serwer, musisz przerwać wykonywanie tej komórki w Colab)
--------------------------------------------------------------------
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
